<a href="https://colab.research.google.com/github/ronaldoaf/999.079_AT_MUDA_STATE_JOB/blob/master/Binance_Kucoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das biblietocas

In [ ]:
!pip install bt
!pip install python-binance

import warnings
import bt
from bt.core import * 
from binance.client import Client as B
from sklearn.linear_model import LinearRegression


warnings.filterwarnings("ignore")
%matplotlib inline



# Download de dados da Binance
 

Série com os código de todos os pares de moedas cotados em **USDT** da Binance ( inclusive os deslistados ) 

In [ ]:
kucoin_futures=["LUNCUSDT","TLMUSDT","EOSUSDT","FILUSDT","IOSTUSDT","ALGOUSDT","CHZUSDT","ENJUSDT","MANAUSDT","DGBUSDT","ICPUSDT","OMGUSDT","IMXUSDT","CELOUSDT","JASMYUSDT","LOOKSUSDT","KDAUSDT","GALUSDT","LUNAUSDT","NEOUSDT","ZECUSDT","LINAUSDT","VRAUSDT","OPUSDT","UNFIUSDT","IOTXUSDT","XEMUSDT","THETAUSDT","ONEUSDT","CROUSDT","NFTUSDT","UNIUSDT","KNCUSDT","LITUSDT","DASHUSDT","HBARUSDT","CELRUSDT","DOGEUSDT","SUSHIUSDT","VETUSDT","AAVEUSDT","XMRUSDT","ENSUSDT","RUNEUSDT","COMPUSDT","CHRUSDT","LINKUSDT","ZILUSDT","EGLDUSDT","LTCUSDT","ATOMUSDT","1INCHUSDT","XRPUSDT","MKRUSDT","LRCUSDT","ARPAUSDT","AUDIOUSDT","FTMUSDT","ETHUSDT","ETCUSDT","AVAXUSDT","ONTUSDT","BTCUSDT","XLMUSDT","ALICEUSDT","BATUSDT","API3USDT","APEUSDT","PEOPLEUSDT","GRTUSDT","BCHUSDT","DUSKUSDT","ADAUSDT","SHIBUSDT","KAVAUSDT","CTSIUSDT","CRVUSDT","ROSEUSDT","DOTUSDT","SXPUSDT","BALUSDT","SANDUSDT","BANDUSDT","MATICUSDT","BNBUSDT","AXSUSDT","SOLUSDT","NYMUSDT","NEARUSDT","FLOWUSDT","YFIUSDT","XTZUSDT","RNDRUSDT","SRMUSDT","KSMUSDT","C98USDT","TRXUSDT","DYDXUSDT","WAVESUSDT","GMTUSDT","YGGUSDT","SLPUSDT","AGLDUSDT","SNXUSDT","XCNUSDT","MIRUSDT","OCEANUSDT","SOSUSDT"]
pares_USDT=pd.Series([s['symbol'] for s in B().get_exchange_info()['symbols'] 
                      if s['symbol'] in kucoin_futures] )
                      #if s['quoteAsset']=='USDT' and 'SPOT' in s['permissions']])
pares_USDT


---

Para cada par baixamos todo histórico diário a partir de 2018-01-01.
São gerados 2 dataframes:
  

*   **precos** : preços de fechamento
*   **volumes** : volumes em USDT



In [ ]:
dfs_preco,dfs_volume=[],[]

for par in pares_USDT:
  df=pd.DataFrame( B().get_historical_klines(par, '30m','2022-05-01') ).astype('float64')
  df['Time'] = pd.to_datetime(df[0], unit='ms')
  df=df.set_index('Time')

  dfs_preco+=[df[[4]].rename(columns={4:par})]
  dfs_volume+=[df[[7]].rename(columns={7:par})]

precos=pd.concat(dfs_preco,axis=1)
volumes=pd.concat(dfs_volume,axis=1)

#ativo artficial de preco=1 e volume muito alto 
#precos['usd'],volumes['usd']=1.0, 1e12

precos

In [ ]:
precos.to_csv('precos.csv')

# Backtest 

In [ ]:
intevalo_longo=pd.DateOffset(days=15)
intervalo_curto=pd.DateOffset(days=5)
intervalo=pd.DateOffset(days=30)
#Define as datas limites para a negociação de cada ativo
#Se a linha seguinte for NA indica o ativo foi deslistado
limites=pd.notna(precos.shift(-7)) 

#Log do Volume dos ultimos n_dias dias maior que percent percentil,  (maior que a mediana)
def vol_maior_que_percent(n_dias,percent):
  return volumes.rolling(n_dias).sum().apply(lambda x: x>np.percentile(x.fillna(0),percent), axis=1)

#Volume médio dos ultimos N1 dias é maior que a dos últimos N2 dias
def vol_medio_maior(N1,N2):
  return volumes.rolling(N1).sum()/N1> volumes.rolling(N2).sum()/N2

def E(condicoes):
  condicao=True
  for cond in condicoes: 
    condicao=np.logical_and(condicao,cond)
  return condicao


momentum=precos.pct_change().rolling(21).sum()+precos.pct_change().rolling(7).sum()


#Une todas as codições
condicao = E([
              limites, 
              sma7<sma21
              ])





s1 = bt.Strategy('s1', [ bt.algos.RunWeekly(),
                        #bt.algos.PrintDate(),
                        bt.algos.SelectHasData(intevalo),
                        bt.algos.SelectWhere(condicao ),
                        bt.algos.WeighEqually(),
                        bt.algos.Rebalance()])

t1 = bt.Backtest(s1, precos,integer_positions=False)

res = bt.run(t1)

#res.display()

res.plot()

In [ ]:
data=pd.DataFrame()

n=1

par='BTCUSDT'


data['x']=precos[par].pct_change(n)
data['y']=precos[par].pct_change(n).shift(-1)
data['z']=precos[par].pct_change(n).shift(-2)
data['w']=precos[par].pct_change(n).shift(-3)
data['q']=precos[par].pct_change(n).shift(-4)
data=data.dropna()

data.corr().x

In [ ]:
precos.pct_change()